<a href="https://colab.research.google.com/github/dharma610/Project/blob/master/SIR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.integrate import odeint
import numpy as np
import pandas as pd
from random import sample
import matplotlib.pyplot as plt
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()

     |████████████████████████████████| 1.0MB 2.7MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364065 sha256=39b78169fe802adddcbdeb520c8e3128987ace81aff0d955736113d27c2de687
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


In [2]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [3]:
def plotsir(t, S, I, R):
  f, ax = plt.subplots(1,1,figsize=(10,4))
  ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
  ax.plot(t, I, 'y', alpha=0.7, linewidth=2, label='Infected')
  ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')
  ax.set_xlabel('Time (days)')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.show();

In [4]:
""" gamma is a list of 1000 random number in (0,1) """

import random
gamma = []
for i in range(1000):
    gamma.append(random.uniform(0, 1))
    


In [5]:
""" beta is twice of each of 1000 gamma values """
beta = []
for i in range(1000):
  beta.append(2*gamma[i])  

In [6]:
""" P, Q, M are the list of S, I, R respectievely calculated for 100 days for each of  1000 sets of parameter (beta,gamma)."""
    
P = []
Q = []
M  = []
for i in range(1000): #Plotted for 2 sets
  N = 1000
  S0, I0, R0 = 999, 1, 0  # initial conditions: one infected, rest susceptible
  t = np.linspace(0, 99, 100) # Grid of time points (in days)
  y0 = S0, I0, R0 # Initial conditions vector
  # Integrate the SIR equations over the time grid, t.
  ret = odeint(deriv, y0, t, args=(N, beta[i], gamma[i]))
  S, I, R = ret.T
  P.append(S)
  Q.append(I)
  M.append(R)

  

In [7]:
""" list to dataftrame """

S_value = pd.DataFrame(P)
I_value = pd.DataFrame(Q)
R_value = pd.DataFrame(M)

In [26]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
# define dataset
s_list = []
i_list = []
r_list = []
s_list1 = []
i_list1= []
r_list1= []
for i in range(1000):
  s_list.append(array(S_value.iloc[i][0:70]))
  i_list.append(array(I_value.iloc[i][0:70]))
  r_list.append(array(R_value.iloc[i][0:70]))


for i in range(1000):
  s_list1.append(array(S_value.iloc[i][70:]))
  i_list1.append(array(I_value.iloc[i][70:]))
  r_list1.append(array(R_value.iloc[i][70:]))  

In [27]:
# reshape series
for i in range(1000):
  s_list[i] =s_list[i].reshape(70,1)
  i_list[i] =i_list[i].reshape(70,1)
  r_list[i] =r_list[i].reshape(70,1)

for i in range(1000):
  s_list1[i] =s_list1[i].reshape(30,1)
  i_list1[i] =i_list1[i].reshape(30,1)
  r_list1[i] =r_list1[i].reshape(30,1)  

In [28]:
p = []
q = []
r = []
p1 = []
q1 = []
r1 = []
for i in range(1000):
  p.append(s_list[i])
  q.append(i_list[i])
  r.append(r_list[i])
for i in range(1000):
  p1.append(s_list1[i])
  q1.append(i_list1[i])
  r1.append(r_list1[i])  



In [29]:
s = tuple(p)
t = tuple(q)
u = tuple(r)

s1 = tuple(p1)
t1 = tuple(q1)
u1 = tuple(r1)

In [34]:
# horizontally stack columns
dataset = hstack(s)
# define generator
n_input = 2
n_features = dataset.shape[1]
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)
# number of samples
print('Samples: %d' % len(generator))

Samples: 68


In [35]:
# horizontally stack columns
dataset1 = hstack(s1)
# define generator
n_input = 2
n_features = dataset.shape[1]
generator1 = TimeseriesGenerator(dataset1, dataset1, length=n_input, batch_size=1)
# number of samples
print('Samples: %d' % len(generator1))

Samples: 28


In [47]:
train = []
# print each sample
for i in range(len(generator)):
  x, y = generator[i]
  train.append(x)
	


In [48]:
test = []
# print each sample
for i in range(len(generator1)):
  x, y = generator1[i]
  test.append(x)
	


In [52]:
train[67]

array([[[202.84589962, 202.846255  , 995.92145751, ..., 202.8458996 ,
         202.84589959, 202.84593225],
        [202.84589961, 202.84617276, 995.85052336, ..., 202.8458996 ,
         202.84589959, 202.84592381]]])

In [25]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1000))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator1, steps_per_epoch=1, epochs=500, verbose=0)

In [53]:
# make a one step prediction out of sample
x_input = train[67].reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[138.296    181.18126  754.4353   745.2052   153.92651  117.607666
  143.25935  189.35825  145.04874  141.35876  657.28314  108.34371
  150.08466  141.32596  629.3311   161.56831  139.8554   187.48482
  204.76076  121.84758  159.53491  161.65225  191.07329  175.72287
  155.6861   234.82674  143.59225  144.15923  127.93743  366.10233
  484.70282  190.65999  408.02222  173.61964  123.62829  134.78964
  313.674    184.97865  101.70715  180.69864  151.02322  625.45197
  708.11487  141.93088  141.39545  707.69     167.9371   189.47392
  175.0791   152.92464  155.6652   707.3489   720.632    150.87503
  232.74574  462.16006  178.46492  180.89359  191.04904  161.07866
  392.24875  618.71826  547.13446  280.6033   161.8739   201.76955
  122.793816 284.37262  178.71318  140.9439   164.90842  141.52162
  174.24866  100.69433  259.32086  129.65085  163.79228  115.41591
  156.4945   129.18767  353.81793  153.91637  123.278    310.7012
  365.35635  184.9939   164.28261  146.14503  742.61456  137.1

In [54]:
test[1]

array([[[202.8458996 , 202.84602362, 995.6317693 , ..., 202.84589959,
         202.84589959, 202.84590947],
        [202.8458996 , 202.84599492, 995.55682215, ..., 202.84589959,
         202.84589959, 202.84590692]]])